# Text 3: Latent Dirichlet allocation
**Internet Analytics - Lab 4**

---

**Group:** *Y*

**Names:**

* *Kristian Aurlien*
* *Mateusz Paluchowski*


---

#### Instructions

*This is a template for part 3 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [1]:
import pickle
import numpy as np
from utils import load_json, load_pkl

from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors

## Exercise 4.8: Topics extraction

Using your pre-processed courses dataset, extract topics using LDA.
    1. Print k = 10 topics extracted using LDA and give them labels. 
    2. How does it compare with LSI?
You can use the default values for all parameters.

In [24]:
courses = load_pkl('courses.pkl')
tf_matrix = load_pkl('tf_matrix.pkl')

In [38]:
#We want word count vectors not colums thus we need to transpose our TF_matrix 
tf_matrix_T = tf_matrix.T

In [39]:
tf_matrix_T.tofile('tf_matrix_T.txt', sep=" ")

In [40]:
data = sc.textFile("tf_matrix_T.txt")
parsedData = data.map(lambda line: Vectors.dense([float(x) for x in line.strip().split(' ')]))
# Index documents with unique IDs`
corpus = parsedData.zipWithIndex().map(lambda x: [x[1], x[0]]).cache()

In [42]:
ldaModel = LDA.train(corpus, k=10)

In [44]:
topics = ldaModel.topicsMatrix()

Py4JJavaError: An error occurred while calling o188.topicsMatrix.
: java.lang.OutOfMemoryError: Java heap space
	at scala.reflect.ManifestFactory$$anon$12.newArray(Manifest.scala:141)
	at scala.reflect.ManifestFactory$$anon$12.newArray(Manifest.scala:139)
	at breeze.linalg.DenseMatrix$.zeros$mDc$sp(DenseMatrix.scala:345)
	at org.apache.spark.mllib.clustering.DistributedLDAModel.topicsMatrix$lzycompute(LDAModel.scala:569)
	at org.apache.spark.mllib.clustering.DistributedLDAModel.topicsMatrix(LDAModel.scala:562)
	at org.apache.spark.mllib.api.python.LDAModelWrapper.topicsMatrix(LDAModelWrapper.scala:30)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.

In [43]:
# Output topics. Each is a distribution over words (matching word count vectors)
print("Learned topics (as distributions over vocab of " + str(ldaModel.vocabSize())
      + " words):")
topics = ldaModel.topicsMatrix()
for topic in range(3):
    print("Topic " + str(topic) + ":")
    for word in range(0, ldaModel.vocabSize()):
        print(" " + str(topics[word][topic]))

Learned topics (as distributions over vocab of 9287250 words):


Py4JJavaError: An error occurred while calling o188.topicsMatrix.
: java.lang.OutOfMemoryError: Java heap space
	at scala.reflect.ManifestFactory$$anon$12.newArray(Manifest.scala:141)
	at scala.reflect.ManifestFactory$$anon$12.newArray(Manifest.scala:139)
	at breeze.linalg.DenseMatrix$.zeros$mDc$sp(DenseMatrix.scala:345)
	at org.apache.spark.mllib.clustering.DistributedLDAModel.topicsMatrix$lzycompute(LDAModel.scala:569)
	at org.apache.spark.mllib.clustering.DistributedLDAModel.topicsMatrix(LDAModel.scala:562)
	at org.apache.spark.mllib.api.python.LDAModelWrapper.topicsMatrix(LDAModelWrapper.scala:30)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.

## Exercise 4.9: Dirichlet hyperparameters

## Exercise 4.10: EPFL's taught subjects

## Exercise 4.11: Wikipedia structure